In [1]:
!nvidia-smi

Wed Jan  7 14:40:42 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q diffusers transformers accelerate safetensors gradio torch


In [2]:
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [5]:
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)

pipe.to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.36.0",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [8]:
def generate_image(prompt, style, steps, guidance):
    style_prompts = {
        "Realistic": "ultra realistic, 4k, high detail",
        "Digital Art": "digital art, trending on artstation",
        "Anime": "anime style, vibrant colors",
        "Cyberpunk": "cyberpunk, neon lights, futuristic",
        "Sketch": "pencil sketch, black and white"
    }

    full_prompt = f"{prompt}, {style_prompts[style]}"

    image = pipe(
        full_prompt,
        num_inference_steps=steps,
        guidance_scale=guidance
    ).images[0]

    return image


In [9]:
with gr.Blocks(css="""
body {background-color: #0f172a;}
.gradio-container {font-family: 'Segoe UI', sans-serif;}
""") as demo:

    gr.Markdown("""
    <h1 style='text-align:center; color:white;'>🎨 AI Image Generator</h1>
    <h3 style='text-align:center; color:#cbd5e1;'>
    Image Generation using Pre-trained Stable Diffusion
    </h3>
    <p style='text-align:center; color:#94a3b8;'>
    Task-02 | Generative AI Internship
    </p>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(
                label="Text Prompt",
                placeholder="A futuristic city at sunset with flying cars"
            )

            style = gr.Dropdown(
                ["Realistic", "Digital Art", "Anime", "Cyberpunk", "Sketch"],
                label="Select Style",
                value="Realistic"
            )

            steps = gr.Slider(10, 50, value=30, step=5, label="Inference Steps")
            guidance = gr.Slider(5, 15, value=7.5, step=0.5, label="Guidance Scale")

            generate_btn = gr.Button("🚀 Generate Image", variant="primary")

        with gr.Column(scale=1):
            output_image = gr.Image(label="Generated Image", height=400)

    generate_btn.click(
        fn=generate_image,
        inputs=[prompt, style, steps, guidance],
        outputs=output_image
    )


/tmp/ipython-input-414362137.py:1: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css="""


In [10]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7f3e062e75696351f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
